# Access ERA5 data from glade and plot temp anomaly for Colorado

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
import intake_esm
import intake
import pandas as pd
import cartopy.crs as ccrs  # Correct import for coordinate reference systems
import cartopy.feature as cfeature
from holoviews import opts
import geoviews as gv
import holoviews as hv

In [2]:
import dask
from dask.distributed import Client, performance_report
from dask_jobqueue import PBSCluster

### Specify global variables

In [3]:
lat = 40.014
lon =  -105.270 + 360
# Bounding box for Colorado
colorado_right  =  (-109.060253 + 360)%360
colorado_left   = (-102.041524 + 360)%360 
colorado_bottom = 37
colorado_top    = 41
###
baseline_year = 1940
current_year  = 2023
day           = 11
month         = 6

In [4]:
rda_scratch = '/gpfs/csfs1/collections/rda/scratch/harshah'

### Spin-up PBS Cluster

In [5]:
# Create a PBS cluster object
cluster = PBSCluster(
    job_name = 'dask-pythia-24',
    cores = 1,
    memory = '4GiB',
    processes = 1,
    local_directory = rda_scratch + '/dask/spill',
    resource_spec = 'select=1:ncpus=1:mem=4GB',
    queue = 'casper',
    walltime = '1:00:00',
    #interface = 'ib0'
    interface = 'ext'
)

/glade/work/harshah/conda-envs/zarr_experiments/lib/python3.11/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 35271 instead
  warnings.warn(


In [6]:
client = Client(cluster)
client

Connection method: Cluster object,Cluster type: dask_jobqueue.PBSCluster
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/harshah/proxy/35271/status,
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/harshah/proxy/35271/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://128.117.208.97:40361,Workers: 0
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/harshah/proxy/35271/status,Total threads: 0
Started: Just now,Total memory: 0 B


### Scale the cluster

In [7]:
cluster.scale(2)
cluster

Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/harshah/proxy/35271/status,Workers: 0
Total threads: 0,Total memory: 0 B
Comm: tcp://128.117.208.97:40361,Workers: 0
Dashboard: https://jupyterhub.hpc.ucar.edu/stable/user/harshah/proxy/35271/status,Total threads: 0
Started: Just now,Total memory: 0 B


### Open ERA5 catalog 

In [8]:
era5_cat = intake.open_esm_datastore('/gpfs/csfs1/collections/rda/transfer/harshah/intake_catalogs/era5_catalog.json')
era5_cat

/glade/work/harshah/conda-envs/zarr_experiments/lib/python3.11/site-packages/intake_esm/cat.py:249: DtypeWarning: Columns (2) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(


,unique
era_id,1
datatype,2
level_type,1
step_type,7
table_code,4
param_code,164
variable,212
long_name,212
units,33
year,85


In [9]:
era5_df = era5_cat.df
era5_df.head()

,era_id,datatype,level_type,step_type,table_code,param_code,variable,long_name,units,year,month,format,frequency,path
0,e5,an,NaN,pl,128,60,PV,Potential vorticity,K m**2 kg**-1 s**-1,1940,1,nc,hourly,/gpfs/csfs1/collections/rda/data/ds633.0/e5.op...
1,e5,an,NaN,pl,128,60,PV,Potential vorticity,K m**2 kg**-1 s**-1,1940,1,nc,hourly,/gpfs/csfs1/collections/rda/data/ds633.0/e5.op...
2,e5,an,NaN,pl,128,60,PV,Potential vorticity,K m**2 kg**-1 s**-1,1940,1,nc,hourly,/gpfs/csfs1/collections/rda/data/ds633.0/e5.op...
3,e5,an,NaN,pl,128,60,PV,Potential vorticity,K m**2 kg**-1 s**-1,1940,1,nc,hourly,/gpfs/csfs1/collections/rda/data/ds633.0/e5.op...
4,e5,an,NaN,pl,128,60,PV,Potential vorticity,K m**2 kg**-1 s**-1,1940,1,nc,hourly,/gpfs/csfs1/collections/rda/data/ds633.0/e5.op...


- Explore the list of variables to find the name of the 2m air temperature

In [10]:
era5_df['variable'].unique()

array(['PV', 'CRWC', 'CSWC', 'Z', 'T', 'U', 'V', 'Q', 'W', 'VO', 'D', 'R',
       'O3', 'CLWC', 'CIWC', 'CC', 'ALUVP', 'ALUVD', 'ALNIP', 'ALNID',
       'CI', 'ASN', 'RSN', 'SSTK', 'ISTL1', 'ISTL2', 'ISTL3', 'ISTL4',
       'SWVL1', 'SWVL2', 'SWVL3', 'SWVL4', 'CAPE', 'LAILV', 'LAIHV',
       'TCLW', 'TCIW', 'SP', 'TCW', 'TCWV', 'STL1', 'SD', 'CHNK', 'MSL',
       'BLH', 'TCC', 'VAR_10U', 'VAR_10V', 'VAR_2T', 'VAR_2D', 'STL2',
       'STL3', 'LCC', 'MCC', 'HCC', 'SRC', 'TCO3', 'IEWS', 'INSS', 'ISHF',
       'IE', 'SKT', 'STL4', 'TSN', 'FAL', 'FSR', 'FLSR', 'LBLT', 'LTLT',
       'LSHF', 'LICT', 'LICD', 'TCRW', 'TCSW', 'U10N', 'V10N', 'VAR_100U',
       'VAR_100V', 'LMLT', 'LMLD', 'VIMA', 'VIT', 'VIKE', 'VITHE',
       'VIPIE', 'VIPILE', 'VITOE', 'VIEC', 'VIMAE', 'VIMAN', 'VIKEE',
       'VIKEN', 'VITHEE', 'VITHEN', 'VIWVE', 'VIWVN', 'VIGE', 'VIGN',
       'VITOEE', 'VITOEN', 'VIOZE', 'VIOZN', 'VILWD', 'VIIWD', 'VIMAD',
       'VIKED', 'VITHED', 'VIWVD', 'VIGD', 'VITOED', 'VIOZD', 'VILWE

- We find that the variable of interest is called 'VAR_2T'

In [11]:
temp_cat = era5_cat.search(variable='VAR_2T',frequency = 'hourly')
temp_cat

,unique
era_id,1
datatype,1
level_type,0
step_type,1
table_code,1
param_code,1
variable,1
long_name,1
units,1
year,85


- Convert catalog to dictonary
- Inspect keys

In [ ]:
dsets = temp_cat.to_dataset_dict()


--> The keys in the returned dictionary of datasets are constructed as follows:
	'datatype.step_type'


In [ ]:
dsets.keys()

- Extract dataset using dictionary

In [ ]:
temp_2m = dsets['an.sfc'].VAR_2T
temp_2m

- Select data corresponding to the dayofyear and colorado for the years baseline_year and current_year

In [ ]:
# Create a pandas Timestamp
current_date  = pd.Timestamp(year=current_year, month=month, day=day)
baseline_date =  pd.Timestamp(year=baseline_year, month=month, day=day)
current_date

In [ ]:
t2m_colorado_current = temp_2m.sel(latitude=slice(colorado_top,colorado_bottom),\
                                         longitude=slice(colorado_right,colorado_left)).sel(time=slice(current_date,current_date+  pd.Timedelta(days=1)))
t2m_colorado_current

In [ ]:
t2m_colorado_baseline = temp_2m.sel(latitude=slice(colorado_top,colorado_bottom),\
                                         longitude=slice(colorado_right,colorado_left)).sel(time=slice(current_date,current_date+  pd.Timedelta(days=1)))
t2m_colorado_baseline

In [ ]:
max_t2m_colorado_anomaly = t2m_colorado_current.max('time') - t2m_colorado_baseline.max('time')
max_t2m_colorado_anomaly

## Let us now plot this data with a map of Colorado in the background

In [ ]:
# Extract the latitude, longitude, and data
latitude = max_t2m_colorado_anomaly['latitude'].values
longitude = max_t2m_colorado_anomaly['longitude'].values
max_t2m_colorado_anomaly = max_t2m_colorado_anomaly.data

In [ ]:
# Create the plot
proj = ccrs.PlateCarree(central_longitude=180)
tiles = gv.tile_sources.ESRI()

# Define the state borders feature
states = cfeature.NaturalEarthFeature(
    category='cultural',
    name='admin_1_states_provinces_lines',
    scale='110m',
    facecolor='none'
)

# Create a GeoViews plot
plot = gv.Image((longitude, latitude, max_t2m_colorado_anomaly), crs=proj, vdims='value').opts(
    cmap='viridis',
    colorbar=True,
    width=800,
    height=600
)

# Add state borders and other features
state_borders = gv.feature.Feature(states, group='State Borders').opts(line_color='black')
# borders = gv.feature.borders.opts(line_dash='dotted', zorder=2)
# state_lines = gv.feature.states.opts(line_color='black', zorder=3)
# borders = gv.feature.borders.opts(line_dash='dotted')
# state_lines = gv.feature.states.opts(line_color='black')

# Combine everything into a single plot
final_plot = tiles * plot * state_borders * borders * state_lines

# Display the plot
hv.extension('bokeh')
final_plot.opts(
    opts.Overlay(projection=ccrs.PlateCarree(central_longitude=180), global_extent=False),
    opts.Image(tools=['hover']),
    opts.Feature(line_width=1.5)
).opts(
    opts.GridSpace(show_title=True, shared_xaxis=True, shared_yaxis=True, height=400, width=600)
)

# To add grid lines with labels, use the options on the plot directly
final_plot.opts(
    opts.Image(xaxis='bottom', yaxis='left', show_grid=True)
)